# Извлечение данных с помощью Gemma 1 2b

## Загрузка и установка Gemma из Kaggle

In [ ]:
!pip install -q -U tf-keras
!pip install -q -U keras-nlp==0.10.0
!pip install -q -U kagglehub>=0.2.4
!pip install -q -U keras>=3

In [ ]:
import os
from google.colab import userdata

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

os.environ["KERAS_BACKEND"] = "jax"
#Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

import keras_nlp
import keras
import csv

print("KerasNLP version: ", keras_nlp.__version__)
print("Keras version: ", keras.__version__)

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

## Подготовка датасета

In [ ]:
!mkdir "dataset"
!mv niikm_data.zip dataset
%cd dataset
!unzip niikm_data.zip
!rm niikm_data.zip
%cd ..

In [ ]:
DATASET_FOLDER = "dataset/"
dataset_file_names = os.listdir(DATASET_FOLDER)

dataset = []
loaded_file_count = 0
ignored_file_count = 0

print("Загрузка датасета...")
for i in dataset_file_names:
    f = open(DATASET_FOLDER + i, 'r', encoding="utf-8")

    #Файлы, не содержащие информацию о ГОСТ или другом нормативном документе, игнорируются.
    buffer = f.readlines()
    ignore = True
    for j in buffer:
        if j.find("ГОСТ") != -1:
            ignore = False
            break
    if ignore:
        print("    INFO: Файл \"" + i + "\" не содержит информацию о ГОСТ. Файл пропущен.")
        ignored_file_count += 1
    else:
        loaded_file_count += 1
        buffer2 = ""
        for j in buffer:
            buffer2 += j
        dataset.append(buffer2)

    f.close()

print("Завершено (" + str(len(dataset_file_names)) + " всего, " + str(loaded_file_count) + " загружено, " + str(ignored_file_count) + " пропущено).")

## Генерация данных

In [ ]:
template = """Ты - ИИ-помощник, созданный для поиска информации о свойствах газа в тексте и её преобразовании в формат JSON.
Оформи ответ на вопрос в соответствии с примером. В ответе должно быть:
- название основного газа в составе;
- название газа;
- химическая формула газа;
- ГОСТ, задающий требования к качеству газа.

ПРИМЕР ВОПРОСА:
НАЗВАНИЕ: Азот жидкий особой чистоты 1 сорт
ГОСТ / НОРМАТИВНЫЙ ДОКУМЕНТ: ГОСТ 9293 - 74
ТРЕБОВАНИЯ К ПРОДУКТУ ПО ГОСТ:
        Наименование показателя                Норма по ГОСТ
        Объёмная доля азота, % не менее                99,999
        Объёмная доля кислорода, % не более                0,0005
        Содержание масла, механических примесей и влаги                выдерживает испытание
        Объёмная доля водорода, % не более                0,0002
        Объёмная доля суммы углеродсодержащих соединений в пересчете на СН4, % не более                0,0003
ОСНОВНЫЕ СВОЙСТВА:
Латинское название:  Nitrogenium
CAS номер: 7727-37-9
UN газа: 1066
UN жидкости: 1977
Код ООН: 1006
Физико-химические свойства:
Электронная конфигурация: 2s22p3
Молекулярая масса: 28.0134
Степень окисления: от +5 до -3
Плотность: 1,251
Удельная теплоёмкость: 1,034
Теплопроводность: 0,026
t кипения: -195,8 °C
t плавления: -210 °C
Внешние признаки: Без цвета вкуса и запаха, химически весьма инертен

ПРИМЕР ОТВЕТА: {"based_on":"на основе Азота","gas_name":"Азот жидкий особой чистоты 1 сорт","gas_letter":"Ar","state_standard":"ГОСТ 9293 - 74"}."""

prompt = template + "\n\nВОПРОС: " + dataset[0] + "\nОТВЕТ:"

generated = gemma_lm.generate(prompt, max_length=1100)
print(generated)

Ты - ИИ-помощник, созданный для поиска информации о свойствах газа в тексте и её преобразовании в формат JSON.
Оформи ответ на вопрос в соответствии с примером. В ответе должно быть:
- название основного газа в составе;
- название газа;
- химическую формулу газа;
- ГОСТ, задающий требования к качеству газа.

ПРИМЕР ВОПРОСА:
НАЗВАНИЕ: Азот жидкий особой чистоты 1 сорт
ГОСТ / НОРМАТИВНЫЙ ДОКУМЕНТ: ГОСТ 9293 - 74
ТРЕБОВАНИЯ К ПРОДУКТУ ПО ГОСТ:
        Наименование показателя                Норма по ГОСТ
        Объёмная доля азота, % не менее                99,999
        Объёмная доля кислорода, % не более                0,0005
        Содержание масла, механических примесей и влаги                выдерживает испытание
        Объёмная доля водорода, % не более                0,0002
        Объёмная доля суммы углеродсодержащих соединений в пересчете на СН4, % не более                0,0003
ОСНОВНЫЕ СВОЙСТВА:
Латинское название:  Nitrogenium
CAS номер: 7727-37-9
UN газа: 1066
UN жидкости

In [ ]:
first_position = generated.find("{", generated.find("{") + 1)
last_position = generated.rfind("}")

print(generated[first_position:last_position + 1])

{"based_on":"на основе водорода","gas_name":"Водород газообразный особо чистый марки В","gas_letter":"H","state_standard":"ТУ 2114-016-78538315-2008"}


In [ ]:
def extract_gas_composition(gemma_lm, sample):
    template = """Ты - ИИ-помощник, созданный для поиска информации о составе газа в тексте.
Оформи ответ на вопрос в соответствии с примером. Каждый пункт ответа должен включать:
- название компонента газа из вопроса;
- химическую формулу компонента газа из вопроса;
- объёмную долю компонента газа из вопроса.

ПРИМЕР ВОПРОСА:
НАЗВАНИЕ: Азот жидкий особой чистоты 1 сорт
ГОСТ / НОРМАТИВНЫЙ ДОКУМЕНТ: ГОСТ 9293 - 74
ТРЕБОВАНИЯ К ПРОДУКТУ ПО ГОСТ:
        Наименование показателя                Норма по ГОСТ
        Объёмная доля азота, % не менее                99,999
        Объёмная доля кислорода, % не более                0,0005
        Содержание масла, механических примесей и влаги                выдерживает испытание
        Объёмная доля водорода, % не более                0,0002
        Объёмная доля суммы углеродсодержащих соединений в пересчете на СН4, % не более                0,0003
ОСНОВНЫЕ СВОЙСТВА:
Латинское название:  Nitrogenium
CAS номер: 7727-37-9
UN газа: 1066
UN жидкости: 1977
Код ООН: 1006
Физико-химические свойства:
Электронная конфигурация: 2s22p3
Молекулярая масса: 28.0134
Степень окисления: от +5 до -3
Плотность: 1,251
Удельная теплоёмкость: 1,034
Теплопроводность: 0,026
t кипения: -195,8 °C
t плавления: -210 °C
Внешние признаки: Без цвета вкуса и запаха, химически весьма инертен

ПРИМЕР ОТВЕТА:
* Азот - N - не менее 99.999%
* Кислород - O - не более 0.0005%
* Водород - H - не более 0.0002%

ВОПРОС:
"""
    #Периодически gemma начинает галлюцинировать. При некоторых вводах она упорно выводит
    #в ответе информацию про оксид азота, даже если в тексте о нём нет ни слова.
    #Проблемы также вызывает непоследовательность в формате таблиц.
    print("    Извлечение состава в виде списка...")

    prompt = template + sample + "\nОТВЕТ: "
    raw_response = gemma_lm.generate(prompt, max_length=1200)
    composition_list = ""
    for i in raw_response[raw_response.rfind("ОТВЕТ:")+6:len(raw_response)].split('*'):
        composition_list += i
    if len(composition_list) == 0:
        print("    ОШИБКА: Gemma не нашла данные о составе продукта.")
        return None

    print("        Завершено.\n    Преобразование списка в запись в формате JSON...")

    template = """Ты - ИИ-помощник, преобразующий списки в данные в формате JSON.
Оформи ответ на вопрос в соответствии с примером.

ПРИМЕР ВОПРОСА:
* Азот - N - не менее 99,999%
* Кислород - O - не более 0,0005%
* Водород - H - не более 0,0002%

ПРИМЕР ОТВЕТА: {"components":[{"name":"Азот","letter":"N","value":"99.999","operation":"не менее"},{"name":"Кислород","letter":"O","value":"0.0005","operation":"не более"},{"name":"Водород","letter":"H","value":"0.0002","operation":"не более"}]}

ВОПРОС:
"""
    prompt = template + composition_list + "\n\nОТВЕТ: "
    #При слишком большом окне модель начинает выводить ответ несколько раз подряд.
    response = gemma_lm.generate(prompt, max_length=500)

    #Магическая последовательность из символов в ответе.
    index = response.find("ОТВЕТ: \n \n ")
    index2 = None
    if index == -1 or index + 11 > len(response) - 1:
        print("    ОШИБКА: Gemma не дала ответ. Возможная причина: слишком мало токенов.")
        return None
    else:
        index += 11
        if response[index] != '{':
            print("    ОШИБКА: Gemma не сгенерировала корректную запись в формате JSON.")
            return None

        #Поиск конца записи в формате JSON и проверка скобок с помощью стека.
        stack = []
        for i in range(index, len(response)):
            if response[i] == '{':
                stack.append('{')
            elif response[i] == '}':
                if len(stack) == 0 or stack[-1] != '{':
                    break
                else:
                    stack.pop(-1)
            elif response[i] == '[':
                stack.append('[')
            elif response[i] == ']':
                if len(stack) == 0 or stack[-1] != '[':
                    break
                else:
                    stack.pop(-1)

            if len(stack) == 0:
                index2 = i
                break

        if index2 == None:
            print("    ОШИБКА: Gemma не сгенерировала корректную запись в формате JSON.")
            return None

    print("        Завершено.")

    return response[index:index2 + 1]

extracted_compositions = []
#файл 24 - хороший пример
for i in range(25,26):
    print("Извлечение состава из файла " + str(i) + "/" + str(len(dataset)) + "...")
    extracted_composition = extract_gas_composition(gemma_lm, dataset[i])
    if extracted_composition == None:
        print("Завершено с ошибкой.")
    else:
        print("Завершено.")
        extracted_compositions.append(extracted_composition)

print(extracted_compositions)

## Парсинг результатов, создание базы и её сохранение в JSON файл

In [ ]:
import json
from copy import deepcopy

#Для теста.
extracted_compositions = ["""{"components":[{"name":"Азот","letter":"N","value":"99.999","operation":"не менее"},{"name":"Кислород","letter":"O","value":"0.0005","operation":"не более"},{"name":"Водород","letter":"H","value":"0.0002","operation":"не более"}]}"""]
extracted_info = [json.loads(i) for i in extracted_compositions]

root_node = {
    "title" : "База технологических газов",
    "code" : " ", #Должен игнорироваться при импорте.
    "path" : "lychev.ms@dvfu.ru / Мой Фонд / Загрузки / База технологических газов$;",
    "date" : " ", #Должен игнорироваться при импорте.
    "creation" : " ", #Должен игнорироваться при импорте.
    "owner_id" : 0, #Должен игнорироваться при импорте.
    "json_type" : "universal",
    "ontology" : "lychev.ms@dvfu.ru / Мой Фонд / Загрузки / Онтология базы технологических газов$;",
    "id" : 17940078395396, #Должен игнорироваться при импорте.
    "name" : "База технологических газов",
    "type" : "КОРЕНЬ",
    "meta" : "Онтология базы технологических газов",
    "successors" : [
        {
            "name" : "Моногазы",
            "type" : "НЕТЕРМИНАЛ",
            "meta" : "Моногазы",
            "successors" : []
        },
        {
            "name" : "Многокомпонентные газовые смеси",
            "type" : "НЕТЕРМИНАЛ",
            "meta" : "Многокомпонентные газовые смеси",
            "successors" : []
        }
    ]
}

component_chemical_formula_node_template = {
    "name":0,
    "type":"НЕТЕРМИНАЛ",
    "meta":"Химическое обозначение",
    "original" : "lychev.ms@dvfu.ru / Мой Фонд / Загрузки / База химических элементов$/Гелий/{:s};",
    "successors" : []
}
terminal_node_template =  {
    "value" : "",
    "type" : "ТЕРМИНАЛ-ЗНАЧЕНИЕ",
    "valtype" : "",
    "meta" : ""
}
non_terminal_node_template = {
    "name" : "",
    "type" : "НЕТЕРМИНАЛ",
    "meta" : "",
    "successors" : []
}

for i in extracted_info:
    for j in range(len(i["components"])):
        component_node = deepcopy(non_terminal_node_template)
        component_node["name"] = j
        component_node["meta"] = "Компонент"

        chemical_formula_node = deepcopy(component_chemical_formula_node_template)
        chemical_formula_node["name"] = i["components"][j]["letter"]
        chemical_formula_node["original"] = chemical_formula_node["original"].format(i["components"][j]["letter"])

        percent_node = deepcopy(terminal_node_template)
        percent_node["value"] = "%"
        percent_node["valtype"] = "STRING"
        percent_node["meta"] = "%"

        operation_node = deepcopy(non_terminal_node_template)
        operation_sign_node = deepcopy(terminal_node_template)
        if i["components"][j]["operation"] == "не менее":
            operation_node["name"] = "Не менее"
            operation_node["meta"] = "Не менее"

            operation_sign_node["value"] = "≥"
            operation_sign_node["valtype"] = "STRING"
            operation_sign_node["meta"] = "≥"
        else:
            operation_node["name"] = "Не более"
            operation_node["meta"] = "Не более"

            operation_sign_node["value"] = "≤"
            operation_sign_node["valtype"] = "STRING"
            operation_sign_node["meta"] = "≤"

        value_node = deepcopy(terminal_node_template)
        value_node["value"] = float(i["components"][j]["value"])
        value_node["valtype"] = "REAL"
        value_node["meta"] = "Числовое значение"

        #Стыковка отношений.
        operation_node["successors"].append(operation_sign_node)
        operation_node["successors"].append(value_node)
        chemical_formula_node["successors"].append(percent_node)
        chemical_formula_node["successors"].append(operation_node)
        component_node["successors"].append(chemical_formula_node)
        root_node["successors"].append(component_node)

print(json.dumps(root_node, indent=4))

# Установка Gemma.cpp (веса с Kaggle, остальное с Github)

In [ ]:
!git clone https://github.com/google/gemma.cpp

In [ ]:
%cd /content/gemma.cpp/
!cmake -B build/
%cd build
!cmake --preset make
!cmake --build --preset make -j 2

In [ ]:
import kagglehub

# path = kagglehub.model_download("google/gemma/gemmaCpp/2b-it-sfp")
# print("Path to model files:", path)

path = kagglehub.model_download("google/gemma/gemmaCpp/7b-it-sfp")
print("Path to model files:", path)

In [ ]:
#%cd /root/.cache/kagglehub/models/google/gemma/gemmaCpp/2b-it-sfp/4
#!mv 2b-it-sfp.sbs /content/2b-it-sfp.sbs
#!mv tokenizer.spm /content/tokenizer.spm

%cd /root/.cache/kagglehub/models/google/gemma/gemmaCpp/7b-it-sfp/3
!mkdir /content/gemmacpp_7b_it_sfp
!mv 7b-it-sfp.sbs /content/gemmacpp_7b_it_sfp/7b-it-sfp.sbs
!mv tokenizer.spm /content/gemmacpp_7b_it_sfp/tokenizer.spm



%cd /content/gemma.cpp/build/

In [ ]:
!./gemma --tokenizer /content/tokenizer.spm --weights /content/2b-it-sfp.sbs --weight_type sfp --model 2b-it

#!./gemma --tokenizer /content/gemmacpp_7b_it_sfp/tokenizer.spm --weights /content/gemmacpp_7b_it_sfp/7b-it-sfp.sbs --weight_type sfp --model 7b-it